In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor,RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_curve, auc,classification_report

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from math import sqrt


plt.rcParams['font.serif'] = ['SimHei']

plt.rcParams['font.sans-serif']='SimHei'
plt.rcParams['axes.unicode_minus']=False # 默认是使用Unicode负号，设置正常显示字符，如正常显示负号

import seaborn as sns
sns.set_style("darkgrid",{"font.sans-serif":['simhei','Droid Sans Fallback']})
import pandas as pd

In [2]:
data=pd.read_csv('A题-全部数据/附件2/M201.csv')
data.head()

日期  时间 生产线编号  物料推送气缸推送状态  物料推送气缸收回状态  物料推送数  物料待抓取数  放置容器数  容器上传检测数  填装检测数  \
0   1   0  M201           0           1      0       0      0        0      0   
1   1   1  M201           1           0      1       0      0        0      0   
2   1   2  M201           0           1      1       0      0        0      0   
3   1   3  M201           1           0      2       0      1        0      0   
4   1   4  M201           0           1      2       1      1        0      0   

   ...  不合格数  物料推送装置故障1001  物料检测装置故障2001  填装装置检测故障4001  填装装置定位故障4002  \
0  ...     0           NaN           NaN           NaN           NaN   
1  ...     0           NaN           NaN           NaN           NaN   
2  ...     0           NaN           NaN           NaN           NaN   
3  ...     0           NaN           NaN           NaN           NaN   
4  ...     0           NaN           NaN           NaN           NaN   

   填装装置填装故障4003  加盖装置定位故障5001  加盖装置加盖故障5002  拧盖装置定位故障6001  拧盖装置拧盖故障6002  
0           NaN           NaN           NaN           NaN           NaN  
1           NaN           NaN           NaN           NaN           NaN  
2           NaN           NaN           NaN           NaN           NaN  
3           NaN           NaN           NaN           NaN           NaN  
4           NaN           NaN           NaN           NaN           NaN  

[5 rows x 37 columns]

In [3]:
data.shape

(7526719, 37)

In [4]:
X =data[[ '时间', '物料推送气缸推送状态', '物料推送气缸收回状态', '物料推送数', '物料待抓取数',
       '放置容器数', '容器上传检测数', '填装检测数', '填装定位器固定状态', '填装定位器放开状态', '物料抓取数', '填装旋转数',
       '填装下降数', '填装数', '加盖检测数', '加盖定位数', '推盖数', '加盖下降数', '加盖数', '拧盖检测数',
       '拧盖定位数', '拧盖下降数', '拧盖旋转数', '拧盖数', '合格数', '不合格数']]

In [5]:
import pandas as pd
import joblib

# 假设 X 已经是一个合适的输入数据格式
# X = ...

# 加载模型
models = {
    "1001": joblib.load('./modellist/Q1_物料推送装置故障1001.bin'),
    "2001": joblib.load('./modellist/Q1_物料检测装置故障2001.bin'),
    "4003": joblib.load('./modellist/Q1_填装装置填装故障4003.bin'),
    "4001": joblib.load('./modellist/Q1_填装装置检测故障4001.bin'),
    "4002": joblib.load('./modellist/Q1_填装装置定位故障4002.bin'),
    "6002": joblib.load('./modellist/Q1_拧盖装置拧盖故障6002.bin'),
    "5002": joblib.load('./modellist/Q1_加盖装置加盖故障5002.bin'),
    "5001": joblib.load('./modellist/Q1_加盖装置定位故障5001.bin'),
    "6001": joblib.load('./modellist/Q1_拧盖装置定位故障6001.bin')
}

# 对新的输入数据 X 进行预测，并收集预测结果
predictions = {}
for code, model in models.items():
    predictions[code] = model.predict(X)

In [6]:

for i,j in predictions.items():
    data[i]=j
    #print(i) 
    print(data[i].value_counts())





1001
0    7522892
1       3827
Name: count, dtype: int64
2001
0    7521726
1       4993
Name: count, dtype: int64
4003
0    7519458
1       7261
Name: count, dtype: int64
4001
0    7520180
1       6539
Name: count, dtype: int64
4002
0    7520115
1       6604
Name: count, dtype: int64
6002
0    7520101
1       6618
Name: count, dtype: int64
5002
0    7523564
1       3155
Name: count, dtype: int64
5001
0    7523933
1       2786
Name: count, dtype: int64
6001
0    7520668
1       6051
Name: count, dtype: int64


In [7]:
a=data[data['1001']==1]

In [8]:
# 根据用户的明确说明，连续的时间值表示持续的故障事件，
# 我们应该相应地将这些事件分组来计算持续时间。

# 提取唯一的日期，以便分别处理每天的数据
days = a['日期'].unique()

# 初始化一个空的DataFrame，用于存储处理后的结果
processed_data = pd.DataFrame(columns=['序号', '日期', '开始时间', '持续时长/秒'])

# 对每一天的数据进行处理，计算连续事件的持续时间
for day in days:
    # 筛选出当前天的数据，并按时间排序
    day_data = a[a['日期'] == day].sort_values(by='时间')
    
    # 初始化变量，用于跟踪连续事件
    start_time = None  # 开始时间
    last_time = None   # 上一个时间点
    duration = 0       # 持续时间
    
    # 用于存储当天事件的列表
    events = []
    
    for index, row in day_data.iterrows():
        if start_time is None:
            # 开始一个新的事件
            start_time = last_time = row['时间']
            duration = 1
        elif row['时间'] == last_time + 1:
            # 继续当前的事件
            last_time = row['时间']
            duration += 1
        else:
            # 结束当前事件并开始一个新的事件
            events.append((day, start_time, duration))
            start_time = last_time = row['时间']
            duration = 1
    
    # 不要忘记添加最后一个事件
    if start_time is not None:
        events.append((day, start_time, duration))
    
    # 将事件转换为DataFrame并追加到处理后的数据中
    day_events_df = pd.DataFrame(events, columns=['日期', '开始时间', '持续时长/秒'])
    day_events_df['序号'] = range(1, len(day_events_df) + 1)
    processed_data = pd.concat([processed_data, day_events_df], ignore_index=True)
    del processed_data['序号']

processed_data

日期   开始时间 持续时长/秒
0     2  22153      4
1    12   9072      6
2    15   5250    157
3    17  20879    163
4    24  25432    152
5    33  10084      7
6    43  28341      9
7    43  28358     65
8    43  28454     36
9    51  14402     13
10   53   1434    155
11  101  24055     14
12  107  16856    164
13  134  20244    157
14  136  24679    154
15  155  19714    159
16  172   9351    154
17  190   4362      1
18  190   6461    155
19  191   4390      4
20  204   4239    155
21  214  15101    167
22  218  13748    165
23  218  14823    159
24  218  19357    155
25  229  26456    155
26  235  11707    161
27  240   2858    154
28  242  18832    162
29  269  18179    159
30  271    657    167
31  289   6485     13
32  306  20720    162
33  338  25183      1
34  353  22530    163

In [9]:

# 统计每个月的故障总次数、最长和最短的持续时长
monthly_stats = processed_data.groupby('日期').agg(
    故障总次数=('开始时间', 'count'),
    最长持续时长=('持续时长/秒', 'max'),
    最短持续时长=('持续时长/秒', 'min')
)

monthly_stats


故障总次数 最长持续时长 最短持续时长
日期                      
2        1      4      4
12       1      6      6
15       1    157    157
17       1    163    163
24       1    152    152
33       1      7      7
43       3     65      9
51       1     13     13
53       1    155    155
101      1     14     14
107      1    164    164
134      1    157    157
136      1    154    154
155      1    159    159
172      1    154    154
190      2    155      1
191      1      4      4
204      1    155    155
214      1    167    167
218      3    165    155
229      1    155    155
235      1    161    161
240      1    154    154
242      1    162    162
269      1    159    159
271      1    167    167
289      1     13     13
306      1    162    162
338      1      1      1
353      1    163    163

In [22]:
shapemax=0
temp=pd.DataFrame()
temp1 = pd.DataFrame()
for d in  ['1001','2001','4001','4002','4003','5001','5002','6001','6002']:
    a=data[data[d]==1]
    # 根据用户的明确说明，连续的时间值表示持续的故障事件，
    # 我们应该相应地将这些事件分组来计算持续时间。

    # 提取唯一的日期，以便分别处理每天的数据
    days = a['日期'].unique()

    # 初始化一个空的DataFrame，用于存储处理后的结果
    processed_data = pd.DataFrame(columns=['序号', '日期', '开始时间', '持续时长/秒'])

    # 对每一天的数据进行处理，计算连续事件的持续时间
    for day in days:
        # 筛选出当前天的数据，并按时间排序
        day_data = a[a['日期'] == day].sort_values(by='时间')

        # 初始化变量，用于跟踪连续事件
        start_time = None  # 开始时间
        last_time = None   # 上一个时间点
        duration = 0       # 持续时间

        # 用于存储当天事件的列表
        events = []

        for index, row in day_data.iterrows():
            if start_time is None:
                # 开始一个新的事件
                start_time = last_time = row['时间']
                duration = 1
            elif row['时间'] == last_time + 1:
                # 继续当前的事件
                last_time = row['时间']
                duration += 1
            else:
                # 结束当前事件并开始一个新的事件
                events.append((day, start_time, duration))
                start_time = last_time = row['时间']
                duration = 1

        # 不要忘记添加最后一个事件
        if start_time is not None:
            events.append((day, start_time, duration))

        # 将事件转换为DataFrame并追加到处理后的数据中
        day_events_df = pd.DataFrame(events, columns=['日期', '开始时间', '持续时长/秒'])
        day_events_df['序号'] = range(1, len(day_events_df) + 1)
        processed_data = pd.concat([processed_data, day_events_df], ignore_index=True)
    if shapemax<processed_data.shape[0]:
        shapemax=processed_data.shape[0]
    del processed_data['序号']
    print(temp.shape)
    temp=pd.concat([temp, processed_data],axis=1)
        # 统计每个月的故障总次数、最长和最短的持续时长
    
    monthly_stats = processed_data.groupby('日期').agg(
        故障总次数=('开始时间', 'count'),
        最长持续时长=('持续时长/秒', 'max'),
        最短持续时长=('持续时长/秒', 'min')
    )
    monthly_stats['month'] = monthly_stats.index // 31 + 1
    monthly_stats1 = monthly_stats.groupby('month').agg(
        故障总次数=('故障总次数', 'count'),
        最长持续时长=('最长持续时长', 'max'),
        最短持续时长=('最短持续时长', 'min')
    )
    temp1=pd.concat([temp1, monthly_stats1],axis=1)
    print(d)
    display(monthly_stats1)
temp1.sort_index(ascending=True)
temp1


(0, 0)
1001


故障总次数 最长持续时长 最短持续时长
month                     
1          5    163      4
2          4    155      7
4          2    164     14
5          2    157    154
6          2    159    154
7          4    167      1
8          5    165    154
9          2    167    159
10         2    162     13
11         1      1      1
12         1    163    163

(35, 3)
2001


故障总次数 最长持续时长 最短持续时长
month                     
1         22      9      1
2         23     19      1
3         21      9      1
4         23      9      1
5         21      8      1
6         23      7      1
7         22      9      1
8         22     19      1
9         23      9      1
10        21     18      1
11        23     19      1
12        16      9      1

(3041, 6)
4001


...

(4424, 21)
6001


故障总次数 最长持续时长 最短持续时长
month                     
1         22     11      1
2         23     11      1
3         21     11      1
4         23      8      1
5         21     14      1
6         23     11      1
7         22     14      1
8         22     11      1
9         23    100      1
10        21     10      1
11        23     11      1
12        16     11      1

(4424, 24)
6002


故障总次数 最长持续时长 最短持续时长
month                     
1          9     11      1
2         14     11      1
3         15     11      1
4         10    168      1
5         12     11      1
6         14    166      1
7         16     11      1
8         11     14      1
9          8     11      1
10        11      8      1
11        13     14      1
12         7    164      1

故障总次数  最长持续时长 最短持续时长 故障总次数  最长持续时长 最短持续时长 故障总次数  最长持续时长 最短持续时长 故障总次数  \
month                                                                        
1       5.0     163      4    22       9      1   3.0     179      1    22   
2       4.0     155      7    23      19      1   3.0     154      1    23   
4       2.0     164     14    23       9      1   3.0     164      1    23   
5       2.0     157    154    21       8      1   NaN     NaN    NaN    21   
6       2.0     159    154    23       7      1   1.0     158      1    23   
7       4.0     167      1    22       9      1   3.0     185      1    22   
8       5.0     165    154    22      19      1   4.0     167      1    22   
9       2.0     167    159    23       9      1   6.0     198      1    23   
10      2.0     162     13    21      18      1   2.0     176    172    21   
11      1.0       1      1    23      19      1   4.0     165      1    23   
12      1.0     163    163    16       9      1   2.0     158      1    16   
3       NaN     NaN    NaN    21       9      1   4.0     156      1    21   

       ... 最短持续时长 故障总次数  最长持续时长 最短持续时长 故障总次数  最长持续时长 最短持续时长 故障总次数  最长持续时长  \
month  ...                                                                  
1      ...      1     7      11      1    22      11      1     9      11   
2      ...      1    11       8      1    23      11      1    14      11   
4      ...      1     9       8      1    23       8      1    10     168   
5      ...      1     9     153      1    21      14      1    12      11   
6      ...      1    13      11      1    23      11      1    14     166   
7      ...      1    14      11      1    22      14      1    16      11   
8      ...      1    10      11      1    22      11      1    11      14   
9      ...      1    10      11      1    23     100      1     8      11   
10     ...      1     9       8      1    21      10      1    11       8   
11     ...      1    13      11      1    23      11      1    13      14   
12     ...      1     6       8      1    16      11      1     7     164   
3      ...      1    12       8      1    21      11      1    15      11   

      最短持续时长  
month         
1          1  
2          1  
4          1  
5          1  
6          1  
7          1  
8          1  
9          1  
10         1  
11         1  
12         1  
3          1  

[12 rows x 27 columns]

In [ ]:
# 新建一列名为“示例列”，并随机赋予一些值
temp.insert(0, '序号', range(1, shapemax+ 1))
temp

In [ ]:
temp.to_excel('result2_M201.xlsx',sheet_name='M201')

In [23]:
temp1.to_excel('result2_M201_month.xlsx',sheet_name='M201')